In [10]:
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [11]:
dataset = {}
dataset['Review'] = []
dataset['Label'] = []

with open('true_reviews.json') as f:
    restaurants = json.load(f)
    for name, review in restaurants.items():
        dataset['Review'].append(review['review'])
        dataset['Label'].append(0)

with open('new_reviews.json') as f:
    restaurants = json.load(f)
    for name, review in restaurants.items():
        dataset['Review'].append(review['review'])
        dataset['Label'].append(1)

In [12]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    dataset['Review'],
    dataset['Label'],
    test_size=0.2,
    random_state=42
)

# Vectorizing the text data using TF-IDF
# You can adjust the max_features parameter based on your dataset size
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Training a logistic regression classifier
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train_tfidf, y_train)

# Making predictions
y_pred = classifier.predict(X_test_tfidf)

# Evaluating the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9285714285714286

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.94      0.94      0.94        18

    accuracy                           0.93        28
   macro avg       0.92      0.92      0.92        28
weighted avg       0.93      0.93      0.93        28



In [1]:
from YelpBot import YelpBot
yelp_bot = YelpBot()

In [2]:
location = "Westwood, Los Angeles"
my_restaurants = yelp_bot.get_list_of_restaurants_from_location(location)

In [3]:
# Get reviews for a restaurant
restaurant_reviews = yelp_bot.get_reviews_from_restaurant(
    restaurant=my_restaurants[0],
    number_of_pages=1
)

Scraping Wolfsglen, page 1


In [4]:
new_review = yelp_bot.generate_new_review(
    restaurant_reviews,
    number_of_reviews=4
)

In [20]:
CLASSIFICATION_PROMPT = """You will be given a review of a restaurant.
Classify the review into one of the following two categories: 
    1. Human-generated
    2. Machine-generated,
Return 0 if the review is Human-generated or 1 if the review is
Machine-generated, and nothing else. MAKE SURE your output is only made of a 0
or a 1.
Restaurant review: {review_to_test}"""

In [14]:
dataset = []
name = next(iter(restaurant_reviews))
for r in restaurant_reviews[name]['reviews'].values():
    dataset.append(r['text'])

name = next(iter(new_review))
dataset.insert(2, new_review[name]['text'])

In [25]:
import os
import json
import numpy as np
from openai import OpenAI
from IPython.display import display, HTML

api_key_path = os.path.join(
    'user_credentials',
    'api_keys.json'
)

with open(api_key_path) as f:
    keys = json.load(f)
    openai_api_key = keys['openai_api_key']

client = OpenAI(api_key=openai_api_key)

In [18]:
def get_completion(
    messages: list[dict[str, str]],
    model: str = "gpt-4",
    max_tokens=500,
    temperature=0,
    stop=None,
    seed=123,
    tools=None,
    # whether to return log probabilities of the output tokens or not. If true, returns the log probabilities of each output token returned in the content of message..
    logprobs=None,
    top_logprobs=None,
) -> str:
    params = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stop": stop,
        "seed": seed,
        "logprobs": logprobs,
        "top_logprobs": top_logprobs,
    }
    if tools:
        params["tools"] = tools

    completion = client.chat.completions.create(**params)
    return completion

In [26]:
for review in dataset:
    API_RESPONSE = get_completion(
        [{
            "role": "user",
            "content": CLASSIFICATION_PROMPT.format(review_to_test=review),
        }],
        logprobs=True,
        top_logprobs=2,
        model="gpt-3.5-turbo",
    )

    top_two_logprobs = API_RESPONSE.choices[0].logprobs.content[0].top_logprobs
    html_content = ""

    for i, logprob in enumerate(top_two_logprobs, start=1):
        linear_prob = np.round(np.exp(logprob.logprob) * 100, 2)
        html_content += (
            f"<span style='color: cyan'>Output token {i}:</span> {logprob.token}, "
            f"<span style='color: darkorange'>logprobs:</span> {logprob.logprob}, "
            f"<span style='color: magenta'>linear probability:</span> {linear_prob}%<br>"
        )
    display(HTML(html_content))